# Scores
Get scores for current games:
pro and college football, pro and college basketball, mlb, nhl

In [9]:
from urllib import request

# league strings
NCAA_FB = 'ncf'
NFL = 'nfl'
MLB = 'mlb'
NBA = 'nba'
NHL = 'nhl'
NCAA_BB = 'mens-college-basketball'

def get_scores(league,team_filter=None):
    # function to retrieve the raw data and clean it up.
    # team_filter can be used to look for a particular team or teams.
    # see examples of calling this function in the comments below.

        scores = {}
        STRIP = "()1234567890 "
        if team_filter:
                team_filter=team_filter.lower().split(', ')

        try:
                #visit espn bottomline website to get scores as html page
                url = 'http://sports.espn.go.com/'+league+'/bottomline/scores'
                #print(url)
                
                req = request.Request(url)
                response = request.urlopen(req)
                page = response.read()

                #url decode the page and split into list
                data = request.unquote(str(page)).split('&'+league+'_s_left')

                #extract the important data
                for i in range(1,len(data)):

                        #get rid of junk at beginning of line, remove ^ which marks team with ball
                        main_str = data[i][data[i].find('=')+1:].replace('^','')

                        #extract time, you can use the ( and ) to find time in string
                        time =  main_str[main_str.rfind('('):main_str.rfind(')')+1].strip()

                        #extract score, it should be at start of line and go to the first (
                        score =  main_str[0:main_str.rfind('(')].strip()

                        #extract espn gameID use the keyword gameId to find it
                        gameID = main_str[main_str.rfind('gameId')+7:].strip()
                                                    
                        
                        if gameID == '':
                                #something wrong happened
                                continue
                          
                        #split score string into each teams string
                        team1_name = ''
                        team1_score = '0'
                        team2_name = ''
                        team2_score = '0'
                        

                        if (' at ' not in score):
                                teams = score.split('  ')
                                team1_name = teams[0][0:teams[0].rfind(' ')].lstrip(STRIP)
                                team2_name = teams[1][0:teams[1].rfind(' ')].lstrip(STRIP)
                                team1_score = teams[0][teams[0].rfind(' ')+1:].strip()
                                team2_score = teams[1][teams[1].rfind(' ')+1:].strip()
                        else:
                                teams = score.split(' at ')
                                team1_name = teams[0].lstrip(STRIP)
                                team2_name = teams[1].lstrip(STRIP)
                                
                        #add to return dictionary
                        if not team_filter:
                                scores[gameID] = ['','','','','']
                                scores[gameID][0] = team1_name
                                scores[gameID][1] = team1_score
                                scores[gameID][2] = team2_name
                                scores[gameID][3] = team2_score
                                scores[gameID][4] = time
                        elif team1_name.lower() in team_filter or team2_name.lower() in team_filter:
                                scores[gameID] = ['','','','','']
                                scores[gameID][0] = team1_name
                                scores[gameID][1] = team1_score
                                scores[gameID][2] = team2_name
                                scores[gameID][3] = team2_score
                                scores[gameID][4] = time
                                
        except Exception as e:
                print(str(e))
                raise e

        return scores

#scores = get_scores(NCAA_FB, 'Ohio State, Georgia Tech, NC State, Notre Dame')
#scores = get_scores(NCAA_FB, '')
#scores = get_scores(NFL,'')
#scores = get_scores(NHL,'')
#scores = get_scores(NBA,'')
scores = get_scores(MLB,'')

game_score = ''

for key,val in scores.items():
        teama  = str(val[0])
        scorea = str(val[1])
        teamb  = str(val[2])
        scoreb = str(val[3])
        time   = str(val[4])
        
        game_score += '{:>16}{:>3}'.format(teama,scorea)+'  '+'{:<3}{:<16}'.format(scoreb,teamb)+' '+'{:>15}\n'.format(time)

games = game_score.split('\n')
num_games = len(games)
scores_per_page = 25
num_pages = int(num_games/scores_per_page) + (num_games % scores_per_page > 0)   # rounds up...

for i in range(num_pages):
    # Take the current 25-game slice.
    start = i * 25
    stop = min((i + 1) * 25, len(games))
    # Grab the slices.
    lines = games[start: stop]

    # Join slices into one string
    msg = '\n'.join(lines)


print(msg)

      Cincinnati  6  5  Chicago Cubs             (FINAL)
           Miami 10  12 Washington               (FINAL)
      LA Dodgers 10  2  Pittsburgh               (FINAL)
       San Diego  6  3  Toronto                  (FINAL)
         Detroit  9  8  NY Mets                  (FINAL)
       Tampa Bay  1  3  Cleveland                (FINAL)
          Boston  3  4  Houston                  (FINAL)
    Philadelphia  6  4  Milwaukee                (FINAL)
     Chicago Sox  4  11 Minnesota                (FINAL)
         Atlanta  5  2  St. Louis                (FINAL)
      NY Yankees  0  0  Kansas City          (POSTPONED)
       Baltimore  6  8  Colorado                 (FINAL)
         Seattle  2  6  Oakland                  (FINAL)
           Texas  4  3  LA Angels                (FINAL)
         Arizona 18  2  San Francisco            (FINAL)

